Imbalanced Dataset and SMOTE

https://www.youtube.com/watch?v=yh2AKoJCV3k

Hyperparameter tuning

using Optuna
https://www.youtube.com/watch?v=E2b3SKMw934&t=29s

Grid search CV and random search CV videos are also pending .

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
df=pd.read_csv('synthetic_imbalanced_dataset.csv')
df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,target
0,1.634005,0.090678,-0.786953,1.172431,2.259654,1.065964,0.017832,-1.542546,-1.446147,0.234793,...,0.857126,0.144281,0.037567,-2.434007,-5.555509,-0.346671,-2.221558,-5.004515,-0.833142,0
1,1.789649,1.088372,-0.230989,2.214075,-2.127782,-0.497370,0.714550,0.066437,-4.808627,0.007713,...,0.694528,-1.436457,0.309054,2.169434,-1.552412,1.481881,-1.861189,-0.666689,0.574386,0
2,-2.355643,2.218601,-1.603269,0.873394,0.401483,0.717264,-0.859399,-1.042190,-2.175965,0.980231,...,0.544434,-2.466258,-0.470256,0.073018,-2.203531,-2.299263,-1.742761,-0.271579,-0.359285,0
3,-1.596198,-0.857427,1.772434,-0.639361,1.419409,-0.438525,0.281949,2.345145,1.006230,0.389135,...,-1.025051,-2.422975,1.579807,-0.300713,4.267120,2.893775,1.236697,6.034785,-0.045711,0
4,2.840049,-2.489600,-0.844902,-1.594362,-4.688517,0.459637,0.913607,-1.143505,1.263937,-2.040928,...,4.176424,1.341742,0.133565,1.743819,1.531188,2.269808,0.053489,-3.151109,1.603702,0


In [8]:
x=df.drop('target',axis=1)
y=df['target']

In [10]:
print('Data shape:',x.shape)
print('Target distribution:\n',y.value_counts())

Data shape: (1000, 20)
Target distribution:
 target
0    843
1    157
Name: count, dtype: int64


### Cross-Validation witth SVM

In [13]:
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.svm import SVC

# SVM with default parameter
svm=SVC()

# Stratified K-Fold
cv=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
scores=cross_val_score(svm,x,y,scoring='f1',cv=cv)

print('F1 scores across folds:',scores)
print('Mean F1 score:',scores.mean())
                  

F1 scores across folds: [0.80769231 0.87272727 0.84210526 0.79245283 0.70588235]
Mean F1 score: 0.8041720053414663


In [15]:
from sklearn.model_selection import GridSearchCV

param_grid={
    'C':[0.1,1,10],
    'kernel':['linear','rbf'],
    'gamma':['scale','auto']
}

grid=GridSearchCV(SVC(),param_grid,scoring='f1',cv=cv,n_jobs=-1) 
grid.fit(x,y)

print('Best parameters:',grid.best_params_)
print('Best F1 score:',grid.best_score_)

Best parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Best F1 score: 0.8749985132322331


#### SMOTE (Synthetic Minority Oversampling Technique)

In [17]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote=SMOTE(random_state=42)
x_res,y_res=smote.fit_resample(x,y)
y.value_counts()

C:\Users\ACER\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\ACER\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\ACER\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ACER\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\ACER\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(

target
0    843
1    157
Name: count, dtype: int64